In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd

In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 778kB 3.5MB/s 
     |████████████████████████████████| 1.1MB 33.3MB/s 
     |████████████████████████████████| 890kB 44.3MB/s 
     |████████████████████████████████| 3.0MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=95b5a229b3f378e2e1f54985472020ff9d751e0430f7bdf66868b733de3011d7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
os.chdir('/content/drive/My Drive/bert')

In [ ]:
# from tokenization import ThaiTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig, BertModel, TFBertForPreTraining, TFBertForClassification
import sentencepiece as spm

ImportError: ignored

In [ ]:
configuration = BertConfig(vocab_size=25004, 
                           hidden_size=768, 
                           num_hidden_layers=12, 
                           num_attention_heads=12, 
                           intermediate_size=3072, 
                           hidden_act='gelu', 
                           hidden_dropout_prob=0.1, 
                           attention_probs_dropout_prob=0.1, 
                           max_position_embeddings=512, 
                           type_vocab_size=2, 
                           initializer_range=0.02)  # default paramters and configuration for BERT

In [ ]:
# Load the fast tokenizer from saved file
# tokenizer = ThaiTokenizer('th.wiki.bpe.op25000.vocab', 'th.wiki.bpe.op25000.model')
# transformers.BertTokenizer('th.wiki.bpe.op25000.vocab', 
#                            do_lower_case=True, 
#                            do_basic_tokenize=True, 
#                            never_split=None, 
#                            unk_token='[UNK]', 
#                            sep_token='[SEP]', 
#                            pad_token='[PAD]', 
#                            cls_token='[CLS]', 
#                            mask_token='[MASK]', 
#                            tokenize_chinese_chars=True, **kwargs)

In [ ]:
with open('th.wiki.bpe.op25000.vocab', 'r', encoding='utf8') as f:
  for ind,i in enumerate(f):
    print(i.split()[0])
    if ind > 10:
      break

[PAD]
[CLS]
[SEP]
[MASK]
<unk>
<s>
</s>
00
อง
ี่
าร
▁เ


In [ ]:
class InputFeatures():
  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id):
    self.input_ids = input_ids
    self.attention_mask = input_mask
    self.token_type_ids = segment_ids
    self.label_id = label_id

class ThaiTokenizer():
  def __init__(self,
               model_dir,
               vocab_dir):
    #<unk> ==> ind = 4
    self.dictionary = {}
    with open(vocab_dir, 'r', encoding='utf8') as f:
      for ind,i in enumerate(f):
        self.dictionary[i.split()[0]] = ind
    self.sp = spm.SentencePieceProcessor(model_file=model_dir)
  def tokenize(self,
                text):
    return self.sp.encode(text, out_type=str)
  def convert_tokens_to_ids(self,
                            tokens):
    tmp = []
    for i in tokens:
      try:
        tmp.append(self.dictionary[i])
      except:
        tmp.append(4)
    return tmp
    

class Text_classification_examples():
  def __init__(self, tokenizer, max_len, ulabels):
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.ulabels = ulabels

  def preprocess(self,
                  texts,
                  labels):
    assert len(texts) == len(labels), 'len(texts) != len(labels)'
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "label_onehot": [],
    }
    for i in range(len(labels)):
      feature_obj = self.convert_single_example(i, texts[i], labels[i])
      for feature in dataset_dict:
        if feature == 'label_onehot':
          dataset_dict['label_onehot'].append(keras.utils.to_categorical(getattr(feature_obj, 'label_id'), num_classes=len(self.ulabels)))
        else:
          dataset_dict[feature].append(getattr(feature_obj, feature))
    return [dataset_dict["input_ids"], dataset_dict["token_type_ids"], dataset_dict["attention_mask"]], [dataset_dict["label_onehot"]]

  def convert_single_example(self, 
                             ex_index, 
                             text_a, 
                             label):
    """Converts a single `InputExample` into a single `InputFeatures`."""

    tokens_a = self.tokenizer.tokenize(text_a)
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > self.max_len - 2:
      tokens_a = tokens_a[:(self.max_len - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)


    input_ids = self.tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < self.max_len:
      input_ids.append(0)
      input_mask.append(0)
      segment_ids.append(0)
    # print(len(input_ids))
    # print(len(input_mask))
    # print(len(segment_ids))
    assert len(input_ids) == self.max_len
    assert len(input_mask) == self.max_len
    assert len(segment_ids) == self.max_len

    label_id = self.ulabels[label]
    if ex_index < 5:
      print("*** Example ***")
      print("tokens: %s" % " ".join(tokens))
      print("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      print("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      print("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
      print("label: %s (id = %d)" % (label, label_id))

    feature = InputFeatures(
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        label_id=label_id)
    return feature

In [ ]:
tokenizer = ThaiTokenizer('th.wiki.bpe.op25000.model', 'th.wiki.bpe.op25000.vocab')
max_len = 128
ulabels = {'negative': 0, 'neutral': 1, 'positive': 2}

data_preb = Text_classification_examples(tokenizer, max_len, ulabels)

In [ ]:
tr = pd.read_csv('/content/drive/My Drive/bert/test_tr.csv',sep=';', names= ['texts', 'labels'])
tr.head()
tr['texts'] = tr['texts'].apply(str)
tr['len'] = tr['texts'].apply(lambda x: len(str(x)))
tr = tr[tr['len']>5].sample(frac=1).reset_index(drop=True)

te = pd.read_csv('/content/drive/My Drive/bert/test_te_with_labels.csv',sep=';', names= ['texts', 'labels'])
te.head()
te['texts'] = te['texts'].apply(str)
te['len'] = te['texts'].apply(lambda x: len(str(x)))
te = te[te['len']>5].sample(frac=1).reset_index(drop=True)

In [ ]:
tr['labels'].value_counts()

negative    1038
positive     996
neutral      913
Name: labels, dtype: int64

In [ ]:
te['labels'].value_counts()

neutral     289
negative     90
positive     70
Name: labels, dtype: int64

In [ ]:
X,Y =data_preb.preprocess(tr['texts'], tr['labels'])
X_te, Y_te=data_preb.preprocess(te['texts'], te['labels'])

*** Example ***
tokens: [CLS] ▁บริการ ดี มาก เลย ค ่ะ [SEP]
input_ids: 1 18247 218 329 3179 24744 22499 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
input_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
label: positive (id = 2)
*** Example ***
tokens: [CLS] ▁คน แบบ ไหน ที่ ควร จะใช้ สิ่ง นี้ยัง ผม อยาก เป็น [SEP]
input

In [ ]:
os.getcwd()

'/content/drive/My Drive/bert'

In [ ]:
# model = keras.models.load_model('.index', custom_objects=None, compile=False)
model = keras.Model()
model.load_weights("./bert_base_th/model.ckpt")

ValueError: ignored

In [ ]:
config = BertConfig.from_json_file('./bert_base_th/bert_config.json')
# encoder = BertForPreTraining.from_pretrained('./bert_base_th/model.ckpt',from_tf=True, config=config)
encoder = TFBertForClassification.from_pretrained('./bert_base_th/model.ckpt', config=config)

NameError: ignored

In [ ]:
type(encoder)

transformers.modeling_bert.BertForPreTraining

In [ ]:
model.summary()

AttributeError: ignored

In [ ]:
max_len = 128
def create_model():
    ## BERT encoder
    # config = BertConfig.from_json_file('./bert_base_th/bert_config.json')
    # encoder = TFBertForPreTraining.from_pretrained('./bert_base_th/model.ckpt.index', from_tf=True, config=config)
    encoder = keras.Model()
    encoder.load_weights("./bert_base_th/model.ckpt")
    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding, _ = encoder(
        input_ids, token_type_ids, attention_mask
    )
    x = layers.Dense(256, activation='relu')(embedding)
    start_logits = layers.Dense(3, name="output", activation='softmax')(x)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=start_logits,
    )
    loss = keras.losses.CategoricalCrossentropy()
    optimizer = keras.optimizers.Adam(lr=5e-5)
    P = tf.keras.metrics.Precision()
    R = tf.keras.metrics.Recall()
    model.compile(optimizer=optimizer, loss=loss, metrics=[P,R,'accuracy'])
    return model

In [ ]:
use_tpu = True
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.39.58.74:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.39.58.74:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


NotImplementedError: ignored

In [ ]:
model.fit(
    X,
    Y,
    epochs=5,  # For demonstration, 3 epochs are recommended
    verbose=2,
    batch_size=32,
    callbacks=[exact_match_callback])